In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from datetime import datetime
# Load the YOLO model
model = YOLO(r'D:\DATA_SCIENCE\PROJECTS\ParkingAutomation\models\YOLOModel.pt')

class PlateDetector:
    def __init__(self, save_dir="captured_plates", conf_threshold=0.85, cooldown_frames=30):
        self.save_dir = save_dir
        self.conf_threshold = conf_threshold
        self.cooldown_frames = cooldown_frames
        self.current_cooldown = 0
        os.makedirs(save_dir, exist_ok=True)
    
    def process_frame(self, frame):
        """Process a single frame and save high-confidence detections"""
        results = model(frame)
        processed_frame = frame.copy()
        
        # Decrease cooldown counter
        if self.current_cooldown > 0:
            self.current_cooldown -= 1
        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                if box.cls == 0:  # License plate class
                    # Get confidence and coordinates
                    conf = float(box.conf[0])
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    
                    # Draw rectangle
                    color = (0, 255, 0) if conf >= self.conf_threshold else (0, 165, 255)
                    cv2.rectangle(processed_frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(processed_frame, f'Conf: {conf:.2f}', (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                    # If confidence meets threshold and not in cooldown period
                    if conf >= self.conf_threshold and self.current_cooldown == 0:
                        # Crop and process plate region
                        plate_region = frame[y1:y2, x1:x2]
                        if plate_region.size > 0:
                            plate_gray = cv2.cvtColor(plate_region, cv2.COLOR_BGR2GRAY)
                            _, plate_thresh = cv2.threshold(plate_gray, 0, 255, 
                                                         cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                            
                            # Save images
                            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                            frame_path = os.path.join(self.save_dir, f"frame_{timestamp}.jpg")
                            plate_path = os.path.join(self.save_dir, f"plate_{timestamp}.jpg")
                            
                            cv2.imwrite(frame_path, frame)
                            cv2.imwrite(plate_path, plate_thresh)
                            
                            print(f"\nDetection above threshold: {conf:.2f}")
                            print(f"Saved frame to: {frame_path}")
                            print(f"Saved plate to: {plate_path}")
                            
                            # Call OCR model
                            # try:
                            #     ocr_result = ocr_model(plate_path)  # This function call is a placeholder
                            # except Exception as e:
                            #     print(f"Error in OCR processing: {e}")
                            
                            # Reset cooldown
                            self.current_cooldown = self.cooldown_frames
        
        # Add threshold and cooldown info to frame
        cv2.putText(processed_frame, f'Threshold: {self.conf_threshold}', 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(processed_frame, f'Cooldown: {self.current_cooldown}', 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        return processed_frame

def capture_and_process():
    # Initialize video capture
    cap = cv2.VideoCapture('../data/car2.mp4')
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    # Initialize detector with confidence threshold
    detector = PlateDetector(conf_threshold=0.85)  # Adjust threshold as needed
    
    print("Starting capture... Press 'q' to stop")
    print(f"Confidence threshold: {detector.conf_threshold}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame")
            break
        
        # Process frame
        processed_frame = detector.process_frame(frame)
        
        # Display frame
        cv2.imshow('License Plate Detection', processed_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    capture_and_process()

Starting capture... Press 'q' to stop
Confidence threshold: 0.85

0: 480x640 1 0, 225.8ms
Speed: 3.4ms preprocess, 225.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

Detection above threshold: 0.86
Saved frame to: captured_plates\frame_20250217_233722.jpg
Saved plate to: captured_plates\plate_20250217_233722.jpg

0: 480x640 1 0, 213.5ms
Speed: 4.9ms preprocess, 213.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 230.0ms
Speed: 2.7ms preprocess, 230.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 201.6ms
Speed: 3.0ms preprocess, 201.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 191.8ms
Speed: 2.9ms preprocess, 191.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 171.5ms
Speed: 2.6ms preprocess, 171.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 160.4ms
Speed: 2.7ms preprocess